# **Korean Summarizer Using Multiple Discriminators**

참조 : https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms

참조 : https://github.com/williamSYSU/TextGAN-PyTorch

* 2020년12월27일 v1.0 완전히 실패...
* 2020년12월27일 오후 Generator 다시 만들고... 역시 실패 한듯... v1.2 완전히 실패...
* 문법 Discriminator를 먼저 학습하고... transfer learning을 사용해 보자.
* 그래도 역시 Generator는 다시 만들어야 할 듯.

In [201]:
DO_ALL = False # 전체 실행하면서 시간 걸리는 걸 Pass 하려면 이걸 False ...


In [202]:

if DO_ALL:
    !pip install sentence-transformers==0.3.0
    !pip install transformers==3.0.2
    !pip install wikipedia
    !pip install konlpy

In [203]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from tensorflow.python.framework import tensor_shape
import keras.utils as ku 

# set seeds for reproducability
from tensorflow.random import set_seed
from numpy.random import seed
set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# 학습을 위한 데이터 준비

네이버 뉴스에서 아무거나 하나 Text를 얻어옴

이것을 '요약' 목표

In [256]:
org_text = """
옛날 어느 집에 귀여운 여자 아기가 태어났어요.

아기는 무럭무럭 자라서, 예쁘고 마음씨 고운 소녀가 되었어요.

그러던 어느날, 소녀의 어머니가 병이들어

그만 세상을 떠나고 말았어요.

 

소녀의 아버지는 홀로 남은 소녀가 걱정되었어요.

그래서 얼마 후 새어머니를 맞이했어요.

새어머니는 소녀보다 나이가 위인 두 딸을 데리고 왔어요.

그러나 새어머니와 언니들은 성질이 고약한 심술쟁이들이었어요.

새어머니는 소녀가 자기 딸들보다 예쁘고 착한 게 못마땅했어요.

 

그런데 이번에는 아버지마저 돌아가셨어요.

소녀는 하녀처럼 하루 종일 쓸고, 닦고, 집안일을 도맡아 했어요.

해도 해도 끝이 없는 집안일이 힘들어 지칠때면

난롯가에 앉아서 잠시 쉬곤 했지요.

 

"엄마, 저애를 신데렐라라고 불러야겠어요."

"온통 재투성이잖아요. 호호호!" 두 언니는 소녀를 놀려 댔어요.

 

어느 날, 왕궁에서 무도회가 열렸어요.

신데렐라의 집에도 초대장이 왔어요.

 

새어머니는 언니들을 데리고 무도회장으로 떠났어요.

신데렐라도 무도회에 가고 싶었어요.

혼자 남은 신데렐라는 훌쩍훌쩍 울기 시작했어요.

"신데렐라, 너도 무도회에 가고 싶니?"

신데렐라가 고개를 들어보니, 마법사 할머니가 빙그레 웃고 있었어요.

 

"내가 너를 무도회에 보내주마

호박 한개와 생쥐 두마리, 도마뱀을 구해 오렴."

  

마법사 할머니가 주문을 외웠어요.

그리고 지팡이로 호박을 건드리자, 호박이 화려한 황금 마차로 변했어요.

이번에는 생쥐와 도마뱀을 건드렸어요.

그랬더니 생쥐는 흰말로, 도마뱀은 멋진 마부로 변했답니다.

 

신데렐라의 옷도 구슬 장식이 반짝이는 예쁜 드레스로 바뀌웠어요.

"신데렐라, 발을 내밀어 보거라."

할머니는 신데렐라에게 반짝반짝 빛나는 유리 구두를 신겨 주었어요

"신데렐라, 밤 열두시가 되면 모든게 처음대로 돌아간단다.

황금 마차는 호박으로, 흰말은 생쥐로, 마부는 도마뱀으로 변하게 돼.

그러니까 바늗시 밤 열두 시가 되기 전에 돌아와야 해. 알겠지?"

 왕자님도 아름다운 신데렐라에게 마음을 빼았겼어요.

완자님은 무도회장에 모인 다른 아가씨들은 쳐다보지도 않고,

신데렐라하고만 춤을 추었어요.

신데렐라는 왕자님과 춤을 추느라 시간 가는 줄도 몰랐어요.

땡, 땡, 땡...... 벽시계가 열두 시를 알리는 소리에 신데렐라는 화들짝 놀랐어요.

 

신데렐라가 허둥지둥 왕궁을 빠져나가는데,

유리 구두 한 짝이 벗겨졌어요.

하지만 구두를 주울 틈이 없었어요.


신데렐라를 뛰쫓아오던 왕자님은 층계에서 유리 구두 한 짝을 주웠어요.

왕자님은 유리 구두를 가지고 임금님께 가서 말했어요.

"이 유리 구두의 주인과 결혼하겠어요."

그래서 신하들은 유리 구두의 주인을 찾아 온 나라를 돌아다녔어요.

 언니들은 발을 오므려도 보고, 구두를 늘려도 보았지만

한눈에 보기에도 유리 구두는 너무 작았어요.

 

그때, 신데렐라가 조용히 다가와 말했어요.

"저도 한번 신어 볼 수 있나요?"

신데렐라는 신하게 건넨 유리 구두를 신었어요,

유리 구두는 신데렐라의 발에 꼭 맞았어요.

 

신하들은 신데렐라를 왕궁으로 데리고 갔어요.

 

그 뒤 신데렐라는 완자님과 결혼하여 오래오래 행복하게 살았대요.



"""

In [255]:
# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n','')
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    return txt 

org_text = clean_text(org_text)
org_text

'옛날 어느 집에 귀여운 여자 아기가 태어났어요. 아기는 무럭무럭 자라서 예쁘고 마음씨 고운 소녀가 되었어요. 그러던 어느날 소녀의 어머니가 병이들어그만 세상을 떠나고 말았어요.  소녀의 아버지는 홀로 남은 소녀가 걱정되었어요. 그래서 얼마 후 새어머니를 맞이했어요. 새어머니는 소녀보다 나이가 위인 두 딸을 데리고 왔어요. 그러나 새어머니와 언니들은 성질이 고약한 심술쟁이들이었어요. 새어머니는 소녀가 자기 딸들보다 예쁘고 착한 게 못마땅했어요.  그런데 이번에는 아버지마저 돌아가셨어요. 소녀는 하녀처럼 하루 종일 쓸고 닦고 집안일을 도맡아 했어요. 해도 해도 끝이 없는 집안일이 힘들어 지칠때면난롯가에 앉아서 잠시 쉬곤 했지요.  엄마 저애를 신데렐라라고 불러야겠어요. 온통 재투성이잖아요.  호호호! 두 언니는 소녀를 놀려 댔어요.  어느 날 왕궁에서 무도회가 열렸어요. 신데렐라의 집에도 초대장이 왔어요.  새어머니는 언니들을 데리고 무도회장으로 떠났어요. 신데렐라도 무도회에 가고 싶었어요. 혼자 남은 신데렐라는 훌쩍훌쩍 울기 시작했어요. 신데렐라 너도 무도회에 가고 싶니?신데렐라가 고개를 들어보니 마법사 할머니가 빙그레 웃고 있었어요.  내가 너를 무도회에 보내주마호박 한개와 생쥐 두마리 도마뱀을 구해 오렴.   마법사 할머니가 주문을 외웠어요. 그리고 지팡이로 호박을 건드리자 호박이 화려한 황금 마차로 변했어요. 이번에는 생쥐와 도마뱀을 건드렸어요. 그랬더니 생쥐는 흰말로 도마뱀은 멋진 마부로 변했답니다.  신데렐라의 옷도 구슬 장식이 반짝이는 예쁜 드레스로 바뀌웠어요. 신데렐라 발을 내밀어 보거라. 할머니는 신데렐라에게 반짝반짝 빛나는 유리 구두를 신겨 주었어요신데렐라 밤 열두시가 되면 모든게 처음대로 돌아간단다. 황금 마차는 호박으로 흰말은 생쥐로 마부는 도마뱀으로 변하게 돼. 그러니까 바늗시 밤 열두 시가 되기 전에 돌아와야 해.  알겠지? 왕자님도 아름다운 신데렐라에게 마음을 빼았겼어요. 완자님은 무도회장에 모인 다른 아가씨들은 쳐다보지도 않고신데렐라

한국어 문법체계에 따라 요약문을 생성하기 위해 한국어 문장 샘플을 준비

'한글 위키백과'에서 임의의 문장을 수집 함

In [206]:
#한국어 위키백과에서 스크랩핑

import wikipedia as wiki
wiki.set_lang('ko')

In [207]:
# '전래동화' 라는 keyword로 100개 page의 Text를 취득

def __search_from_wiki(question,max_rank):
    results = wiki.search(question,results=max_rank)
    print(results)
    contents = []
    for result in results:
        try:
            page = wiki.page(result)
            #print(f"Top wiki result: {page}")
            text = page.content
            ln = len(text)
            print(f'Collecting page : {page} , text length {str(ln)}')
            #if ln < 4000:
            #  contents.append(text)
            #else:
            #  contents.append(text[0:4000])
            contents.append(text)
        except Exception as ex:
            print(ex)
    return contents

if DO_ALL:
    ko_grammar_set_raw = __search_from_wiki("전래동화", 100)

print(f'전체 수집한 Page Count : {len(ko_grammar_set_raw)}')

전체 수집한 Page Count : 197


In [208]:

if DO_ALL:
    ko_grammar_set_raw += __search_from_wiki("역사", 100)

print(f'전체 수집한 Page Count : {len(ko_grammar_set_raw)}')

전체 수집한 Page Count : 197


In [209]:
# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n','')
    txt = txt.replace('=','')    
    return txt 

ko_grammar_set_raw = [clean_text(x) for x in ko_grammar_set_raw]
print('Sample text : ')
print('--------------------------------------------------------------------------------------------')
print(ko_grammar_set_raw[50])
print('--------------------------------------------------------------------------------------------')

Sample text : 
--------------------------------------------------------------------------------------------
백조의 호수(러시아어: Лебединое озеро)는 러시아의 차이콥스키가 작곡한 발레 음악이자 이 음악에 맞춰 공연되는 발레 작품이다. 초연 때는 연출이나 무대 장치가 서툴렀기 때문에 호응을 얻지 못했으나, 차이콥스키의 음악과 프티파, 이바노프(Lev Ivanov)의 개작으로 처음으로 그 진가가 인정되고, 고전 발레를 대표하는 작품으로서 각국에서 상연되기에 이르렀다. 시나리오는 러시아 전래 동화를 기반으로 하여, 악한 마법사의 저주에 걸려 백조로 변한 오데트 공주에 대한 이야기이다. 전체에 넘쳐흐르는 감미로운 선율은 오데트 공주의 슬픈 운명을 묘사하였고, 또한 제3막의 궁정 무도회의 성격(character) 무용에서는 민족적 리듬을 지닌 소곡을 차례로 전개한다. 오데트 공주의 주제 음악은 특히 유명하다. 관습적으로 오데트 공주를 연기하는 프리마 발레리나는 지그프리드 왕자를 유혹하는 흑조 오딜도 동시에 연기한다. 줄거리  제1막 지그프리드 왕자는 교사, 친구, 신민과 함께 그의 생일을 축하한다. 여황은 왕자의 자유로운 생활을 걱정하는 지그프리드의 어머니인 여왕에 의해 곧 방해받고 만다. 여왕은 다음 저녁에 열리는 왕실 무도회에서 신부를 정해야 한다고 당부하지만, 왕자는 사랑을 위해 결혼할 수 없다는 것에 화가 난다. 왕자의 친구인 벤노와 교사는 왕자의 기분을 풀어 주려 한다.  저녁이 드리워질 때 벤노는 백조 무리가 머리 위로 지나가는 것을 보고 사냥을 가자고 제안한다. 왕자와 친구들을 석궁을 들고 백조를 쫓기 위해 떠난다. 제2막 지그프리드 왕자는 친구들과 떨어진다. 왕자가 호수 옆 빈터에 다다랐을 때 백조 무리가 근처에 앉는다. 왕자는 백조를 향해 석궁을 겨누지만 백조 무리 중 한 마리가 아름다운 아가씨 오데트로 변신하는 것을 보자 얼어붙고 만다. 오데트는 처음에 

문장으로 잘라 낸다

In [210]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [211]:
#Split the document into sentences
ko_grammar_sentences = []
for document in ko_grammar_set_raw:
    ko_grammar_sentences += nltk.sent_tokenize(document)

print("Num sentences:", len(ko_grammar_sentences))

Num sentences: 13551


In [212]:
ko_grammar_sentences[300]

"현재 기탄교육은 학습지로는 '기탄국어, 한글떼기, 기탄한글, 기탄수학, 기탄사고력수학, 영단어 암기 끝!, 기탄 한자 빨리따기, 기탄중국어' 등이 있다."

In [213]:
f = open("ko_sentence_set.txt", "a")
for t in ko_grammar_sentences:
    f.write(t+'\n')
f.close()

형태소 분리하여 모든 문장을 형태소 Code로 변환 한다.

In [214]:
from konlpy.tag import Twitter
twitter = Twitter()

print(twitter.pos(ko_grammar_sentences[305]))

[('백운호수', 'Noun'), ('에서', 'Josa'), ('의왕시', 'Noun'), ('와', 'Josa'), ('의왕시', 'Noun'), ('축제', 'Noun'), ('추진', 'Noun'), ('위원회', 'Noun'), ('가', 'Josa'), ('공동', 'Noun'), ('으로', 'Josa'), ('주최', 'Noun'), ('하여', 'Verb'), ('매년', 'Noun'), ('9월', 'Number'), ('에', 'Foreign'), ('이틀', 'Noun'), ('동안', 'Noun'), ('열리는', 'Verb'), ('축제', 'Noun'), ('이다', 'Josa'), ('.', 'Punctuation')]


In [215]:
# 형태소 Code table의 구성

_MAX_MORP_LENGTH = 128
_PADDING_CODE = 0  # padding code
_MISMATCH_CODE = 1 # mismatch word code               ex) @@@
_MISMATCH_WORD = '@@@' # 이거 아래에서 쓴다.

morpheme_table = {}
morp_code = _MISMATCH_CODE+1
morpheme_table['Pad'] = _PADDING_CODE 
morpheme_table['Mst'] = _MISMATCH_CODE 
for sentence in ko_grammar_sentences[:1000]:
    morphemes = twitter.pos(sentence)
    for (word,morp) in morphemes:
        if morp in morpheme_table:
            pass
        else:
            morpheme_table[morp] = morp_code
            morp_code += 1

In [216]:
print('Korean morpheme code table')
print('----------------------------------------------------------')
print('  Morpheme        Code')
print('')
for morp in morpheme_table.keys():
    print(f' {morp.ljust(15)}   {morpheme_table[morp]}')
print('----------------------------------------------------------')


Korean morpheme code table
----------------------------------------------------------
  Morpheme        Code

 Pad               0
 Mst               1
 Noun              2
 Punctuation       3
 Foreign           4
 Josa              5
 Verb              6
 Modifier          7
 Adjective         8
 Suffix            9
 Adverb            10
 Number            11
 Alpha             12
 Conjunction       13
 Determiner        14
 VerbPrefix        15
 Exclamation       16
 KoreanParticle    17
 Eomi              18
 ScreenName        19
 URL               20
----------------------------------------------------------


In [217]:
# morpheme 코드 변환기
def morpheme_encode(sentence):
    encode=[]
    morphemes = twitter.pos(sentence)
    for (word,morp) in morphemes:
        encode.append(_MISMATCH_CODE if word==_MISMATCH_WORD else morpheme_table[morp])
    return encode

code = morpheme_encode('야당이 수단으로 야당에게는 이라며 윤석열 허물어뜨렸고 수사를 군사작전을 하는 시비를 민주당 의원이 국민적 세워 화신이 나라 법위에 우려했다 될 이라며 운영할지 바람 짓밟지만 양자역학 양자역학 걱정하느냐고 누가 청와대 한다 엄포를 양자역학 양자역학 양자역학 편을 파렴치와 난관에 풀이다 대통령이 양자역학 저지하려다가')
print(f'Code length : {len(code)}')

Code length : 67


In [218]:
# 전체 형태소 코드로 변환
if DO_ALL:
    ko_grammar_set = []
    for sentence in ko_grammar_sentences:
        code = morpheme_encode(sentence)
        if len(code) <= _MAX_MORP_LENGTH:
            ko_grammar_set.append(code + [_PADDING_CODE for i in range(_MAX_MORP_LENGTH-len(code))])

    ko_grammar_set = np.asarray(ko_grammar_set)
ko_grammar_set.shape

(13352, 128)

In [219]:
from numpy import savetxt
# save to csv file
savetxt('ko_morpheme_set.csv', ko_grammar_set, delimiter=',')


# Dataset 전체 구성

In [220]:
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer

if DO_ALL:
    # embedder download...
    embedder = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')

In [221]:
BATCH_SIZE = 64
BATCH_COUNT = 50

In [222]:
# 원문의 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n','')
    txt = txt.replace('=','')    
    txt = txt.replace('\"','')    
    txt = txt.replace('\'','')    
    return txt 

org_text = clean_text(org_text)

In [223]:
org_text

'옛날 어느 집에 귀여운 여자 아기가 태어났어요.아기는 무럭무럭 자라서 예쁘고 마음씨 고운 소녀가 되었어요.그러던 어느날 소녀의 어머니가 병이들어그만 세상을 떠나고 말았어요. 소녀의 아버지는 홀로 남은 소녀가 걱정되었어요.그래서 얼마 후 새어머니를 맞이했어요.새어머니는 소녀보다 나이가 위인 두 딸을 데리고 왔어요.그러나 새어머니와 언니들은 성질이 고약한 심술쟁이들이었어요.새어머니는 소녀가 자기 딸들보다 예쁘고 착한 게 못마땅했어요. 그런데 이번에는 아버지마저 돌아가셨어요.소녀는 하녀처럼 하루 종일 쓸고 닦고 집안일을 도맡아 했어요.해도 해도 끝이 없는 집안일이 힘들어 지칠때면난롯가에 앉아서 잠시 쉬곤 했지요. 엄마 저애를 신데렐라라고 불러야겠어요.온통 재투성이잖아요. 호호호! 두 언니는 소녀를 놀려 댔어요. 어느 날 왕궁에서 무도회가 열렸어요.신데렐라의 집에도 초대장이 왔어요. 새어머니는 언니들을 데리고 무도회장으로 떠났어요.신데렐라도 무도회에 가고 싶었어요.혼자 남은 신데렐라는 훌쩍훌쩍 울기 시작했어요.신데렐라 너도 무도회에 가고 싶니?신데렐라가 고개를 들어보니 마법사 할머니가 빙그레 웃고 있었어요. 내가 너를 무도회에 보내주마호박 한개와 생쥐 두마리 도마뱀을 구해 오렴.  마법사 할머니가 주문을 외웠어요.그리고 지팡이로 호박을 건드리자 호박이 화려한 황금 마차로 변했어요.이번에는 생쥐와 도마뱀을 건드렸어요.그랬더니 생쥐는 흰말로 도마뱀은 멋진 마부로 변했답니다. 신데렐라의 옷도 구슬 장식이 반짝이는 예쁜 드레스로 바뀌웠어요.신데렐라 발을 내밀어 보거라.할머니는 신데렐라에게 반짝반짝 빛나는 유리 구두를 신겨 주었어요신데렐라 밤 열두시가 되면 모든게 처음대로 돌아간단다.황금 마차는 호박으로 흰말은 생쥐로 마부는 도마뱀으로 변하게 돼.그러니까 바늗시 밤 열두 시가 되기 전에 돌아와야 해. 알겠지? 왕자님도 아름다운 신데렐라에게 마음을 빼았겼어요.완자님은 무도회장에 모인 다른 아가씨들은 쳐다보지도 않고신데렐라하고만 춤을 추었어요.신데렐라는 왕자님과 춤을 추느라 시

In [224]:
#dataset 다시 만듦

org_text_emb = embedder.encode([org_text])[0]
print(f'Text embedding shape : {org_text_emb.shape}')
dataset = []
for i in range(BATCH_COUNT):
    emb_batch_set = []
    cod_batch_set = []
    for j in range(BATCH_SIZE):
        emb_batch_set.append(org_text_emb)
        cod_batch_set.append(ko_grammar_set[BATCH_SIZE*i+j])

    emb_batch_set = np.asarray(emb_batch_set)
    cod_batch_set = np.asarray(cod_batch_set)
    dataset.append((emb_batch_set,cod_batch_set))

print(f'Total dataset count :{BATCH_COUNT*BATCH_SIZE}')


Text embedding shape : (1024,)
Total dataset count :3200


In [225]:
# 30번째 배치의 형태소코드셋 중 20번째꺼 확인
print(dataset[30][1][20])
print(dataset[31][1][20])

[2 9 2 5 2 2 5 6 2 2 5 6 2 2 5 2 5 6 3 2 5 2 5 2 6 2 5 6 2 5 6 6 2 8 3 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 2  2  5  2  5  2  5  2  2  2  5  2  8  3  2  5  2  6  6  2  5  2  5  3
 11  3 11  3 11  3 11  3 11  3 11  3 11  3  2  2  5  2  5  6  2  6  2  5
  6  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


# Generator 구현

In [226]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([org_text])
total_words = len(tokenizer.word_index)

print(f'Total token count of origin text : {total_words}')

Total token count of origin text : 275


원문을 40개의 token 으로 구성된 문장으로 요약 하는것....<br>
향후 LSTM으로 G 구성 필요

In [227]:

org_term_set = org_text.split(' ')

_MAX_GEN_TOKEN = 40
_NOISE_DIM = len(org_term_set)

word_table = {}

for index, word in zip(range(len(org_term_set)),org_term_set):
    word_table[index] = word

current_token_len = len(word_table)



In [228]:
print('Token table of origin text')
print('---------------------------------------------')
print(' Code         Token      ')
print('')
for k in word_table.keys():
  print( f'  {str(k).ljust(8)}    {word_table[k]}')
print('---------------------------------------------')


Token table of origin text
---------------------------------------------
 Code         Token      

  0           옛날
  1           어느
  2           집에
  3           귀여운
  4           여자
  5           아기가
  6           태어났어요.아기는
  7           무럭무럭
  8           자라서
  9           예쁘고
  10          마음씨
  11          고운
  12          소녀가
  13          되었어요.그러던
  14          어느날
  15          소녀의
  16          어머니가
  17          병이들어그만
  18          세상을
  19          떠나고
  20          말았어요.
  21          소녀의
  22          아버지는
  23          홀로
  24          남은
  25          소녀가
  26          걱정되었어요.그래서
  27          얼마
  28          후
  29          새어머니를
  30          맞이했어요.새어머니는
  31          소녀보다
  32          나이가
  33          위인
  34          두
  35          딸을
  36          데리고
  37          왔어요.그러나
  38          새어머니와
  39          언니들은
  40          성질이
  41          고약한
  42          심술쟁이들이었어요.새어머니는
  43          소녀가
  44          자기
  45          딸들보다
  46          예쁘고
  47        

In [229]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input,
                                     Dense, 
                                     BatchNormalization, 
                                     LeakyReLU,
                                     Softmax,
                                     Reshape, 
                                     Conv2DTranspose,
                                     Conv2D,
                                     Dropout,
                                     Flatten,
                                     Concatenate,
                                     Lambda)
import matplotlib.pyplot as plt

noise를 token_table을 통해 text로 변환하는 변환기 구현

In [230]:

b = tf.constant([1,2,3,4,5,6,11,8,9,10])
c = tf.sort(b,axis=-1,direction='DESCENDING')
wh = tf.where(tf.math.greater_equal(b,c[5-1]))[:5]
wh

<tf.Tensor: shape=(5, 1), dtype=int64, numpy=
array([[5],
       [6],
       [7],
       [8],
       [9]])>

In [231]:
import sys

def to_text(w):

    #tf.print(w)
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            text = ""
            wh = tf.where(tf.math.greater_equal(w_0r,s_w_0r[_MAX_GEN_TOKEN]))[:_MAX_GEN_TOKEN]

            for k in tf.reshape(wh,(_MAX_GEN_TOKEN,)).numpy():
                try:
                    text += word_table[k]+' '
                except Exception as ex:
                    tf.print('to_text : word_table' + str(ex),k)
                
            texts.append(text)
    except Exception as ex:
        tf.print('to_text : ' + str(ex),sys.exc_info())

    return tf.constant(texts,dtype=tf.string)

In [232]:
# to_text 함수의 test
w = tf.random.normal([3,_NOISE_DIM])
print(w.shape)
e = to_text(w)
for t in e:
    print(t.numpy().decode('utf-8'))

(3, 301)
집에 떠나고 아버지는 소녀보다 소녀가 딸들보다 돌아가셨어요.소녀는 집안일을 쉬곤 했지요. 신데렐라라고 어느 열렸어요.신데렐라의 데리고 시작했어요.신데렐라 보내주마호박 두마리 주문을 화려한 생쥐와 마부로 신데렐라에게 반짝반짝 열두시가 돌아간단다.황금 바늗시 되기 돌아와야 무도회장에 모인 가는 빠져나가는데유리 언니들은 오므려도 구두를 보았지만한눈에 구두는 신데렐라가 수 있나요?​신데렐라는 
떠나고 말았어요. 새어머니를 착한 앉아서 댔어요. 집에도 시작했어요.신데렐라 가고 빙그레 웃고 무도회에 주문을 도마뱀을 도마뱀은 변했답니다. 발을 반짝반짝 주었어요신데렐라 모든게 해. 알겠지? 왕자님도 춤을 왕자님과 시를 놀랐어요. 구두를 구두 주웠어요.왕자님은 말했어요.이 신하들은 유리 구두를 늘려도 보기에도 신어 꼭 신데렐라를 뒤 
소녀가 소녀의 아버지마저 했어요.해도 끝이 호호호! 왕궁에서 집에도 떠났어요.신데렐라도 너도 내가 구해  할머니가 주문을 호박을 도마뱀을 신데렐라의 옷도 구슬 예쁜 되면 돼.그러니까 되기 쳐다보지도 신데렐라가 구두 짝이 주울 층계에서 한 신하들은 돌아다녔어요. 보기에도 건넨 구두를 신었어요유리 신데렐라의 발에 그 


생성된 text의 embedding 변환기 구현<br>
embedding은 org_text의 embedding과 비교하여 원문과 유사하게 민들기 위한 목적

In [233]:
# 이거 이틀걸림...잘 몰라서 ㅈㄴ 헤맴

@tf.custom_gradient
def to_embedding(w):

    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 10, 1e-3)  # gradient 증폭 
        dy_arr = tf.reshape(dy,(dy.shape[0],1024,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],w.shape[1]),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],w.shape[1]))
        return dy_arr_st

    #print(w)    
    texts = []
    value = None
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            text = ""
            wh = tf.where(tf.math.greater_equal(w_0r,s_w_0r[_MAX_GEN_TOKEN]))[:_MAX_GEN_TOKEN]
            for k in tf.reshape(wh,(_MAX_GEN_TOKEN,)).numpy():
                text += word_table[k]+' '
                 
            texts.append(text)                 
        value = tf.constant(embedder.encode(texts,show_progress_bar=False),dtype=tf.float64)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad

In [234]:
# to_embedding 함수의 test
e = to_embedding(w)
for t in e:
    print(t.numpy())

[ 0.20697494 -0.16078305  0.04480495 ... -0.18544388  0.67773646
  0.60726237]
[ 0.35158053 -0.53133172  0.05704678 ... -0.37279111  0.59129167
  0.58416045]
[ 0.19844982 -0.38411286  0.00347876 ... -0.39883673  0.88680857
  0.72472489]


In [235]:
# to_compression_ratio

@tf.custom_gradient
def to_compression_ratio(w):
    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 2, 1e-3)  # gradient 증폭, 이래도 되는 건지....
        dy_arr = tf.reshape(dy,(dy.shape[0],1,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],w.shape[1]),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],w.shape[1]))        
        return dy_arr_st

    const = tf.math.reduce_sum(tf.constant([x for x in range(tf.size(w[0]))],tf.int64)) 
    #print(const)
    value = None
    compression_ratio = [] #tf.Variable()
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            wh = tf.where(tf.math.greater_equal(w_0r,s_w_0r[_MAX_GEN_TOKEN]))[:_MAX_GEN_TOKEN]
            cr = tf.math.reduce_sum(tf.reshape(wh,(_MAX_GEN_TOKEN,)))
            compression_ratio.append(cr.numpy()/const.numpy())
        #print(compression_ratio)
        value = tf.constant(compression_ratio,dtype=tf.float64)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad
    

In [236]:
# to_compression_ratio 함수의 test
e = to_compression_ratio(w)
for t in e:
    print(t.numpy())

0.12790697674418605
0.15193798449612403
0.14815060908084163


생성된 text의 morpheme code 변환기 구현<br>
morpheme code는 한국어 문장들(dataset)의 morpheme code와 비교하여 한국어 문법에 가깝게 만들기 위한 목적

In [237]:

@tf.custom_gradient
def to_morpcoding(w):

    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 2, 1e-3)  # gradient 증폭, 이래도 되는 건지....
        dy_arr = tf.reshape(dy,(dy.shape[0],_MAX_MORP_LENGTH,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],w.shape[1]),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],w.shape[1]))
        return dy_arr_st

    #print(w)    
    codes = []
    value = None
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            text = ""
            wh = tf.where(tf.math.greater_equal(w_0r,s_w_0r[_MAX_GEN_TOKEN]))[:_MAX_GEN_TOKEN]
            for k in tf.reshape(wh,(_MAX_GEN_TOKEN,)).numpy():
                text += word_table[k]+' '
                 
            texts.append(text)    
            
        for sentence in texts:
            code = morpheme_encode(sentence)
            if len(code) <= _MAX_MORP_LENGTH:
                codes.append(code + [_PADDING_CODE for i in range(_MAX_MORP_LENGTH-len(code))])
            else:
                codes.append(code[:_MAX_MORP_LENGTH])
        value = tf.constant(codes,dtype=tf.int32)
    except Exception as ex:
        tf.print('to_morpcoding :' + str(ex),sys.exc_info())

    return value, grad

In [238]:
# to_morpcoding 함수의 test
e = to_morpcoding(w)
for t in e:
    print(t.numpy())

[ 2  5  6  2  5  2  5  2  5  2  9  5  6  3  2  5  2  5  6  6  3  2  5 10
  6  3  2  5  2  5  2  6  3  2  6  2 14  2  2  5  8  2  5  2  5  2  5 10
  2  5  6  3  2  2  6  6  2  9  5  6  6  6  2  2  9  5  6  2  5  6  2  5
  2  5  2  5  2  8  3  4  2  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
[ 6  6  3  2  5  8  6  6  3  2  5  2  6  3  2  6  2  6  2  5  2  5  2  5
  2  5  8  3  2  5 10  6  2  2  5  2  3  6  3  2  9  5  2  5  2  9  5  8
  6  3  2  5  2  6  3  2  9  5  2  6  3  2  2  9  5  2  2  5  6  2  5  2
  2  2  5  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
[ 2  5  2  5  2  5  6  3  2  2  5  2  2  3  2  5  2  5  6  3  2  5  2  5
  2  5  2  2  5  2  5  2  5  2  5  2  5  2  5  2  8  6  6  3 10  6  6  2
  5  2  2  5  6  2  5  6  2  9  5  6  3  2  5  6  2  5  6  2  2  5  2  5

Network 구성을 위해 사용자 정의 Layer 를 구현

In [239]:
# 이것도 잘 몰라서 하루 걸림... ㅜㅜ

from keras import backend as K
from keras.layers import Layer

#tf.executing_eagerly()

class Post_processing(Layer):

    def __init__(self, output_dim, encoder_func=None,Tout=tf.float64, **kwargs):
        self.output_dim = output_dim
        self.encoder = encoder_func
        self.Tout = Tout
        super(Post_processing, self).__init__(**kwargs)
    '''
    def build(self, input_shape):
        tf.print('build',input_shape)
        # 이 레이어에 대해 학습가능한 가중치 변수를 만듭니다.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(Post_processing, self).build(input_shape)  # 끝에서 꼭 이 함수를 호출하십시오
    '''
    def call(self, input_data):
        #tf.print('Post_processing : call input_data',input_data.shape)
        value = tf.py_function(self.encoder,[input_data],Tout=self.Tout,name='encode_func')
        #print('value.shape:',value.shape)
        #value.set_shape((input_data.shape[0],self.output_dim))
        if self.output_dim > 0:
            value.set_shape((input_data.shape[0],self.output_dim))
        else:
            value.set_shape((input_data.shape[0],))
        #return tf.reshape(value,[input_data.shape[0]])  

        #value = tf.Variable((tf.zeros([input_data.shape[0],1024]) if self.Tout==tf.float64 else tf.zeros([input_data.shape[0],])),dtype=self.Tout,shape=( (input_data.shape[0],1024) if self.Tout==tf.float64 else (input_data.shape[0],)))
        #tf.py_function(self.encoder,[input_data],Tout=self.Tout)
        return value

    def compute_output_shape(self, input_shape):
        tf.print('compute_output_shape:',input_shape)
        input_shape = tensor_shape.TensorShape(input_shape).as_list()
        if self.output_dim > 0:
            return tensor_shape.TensorShape([input_shape[0], self.output_dim])
        return tensor_shape.TensorShape([input_shape[0]])

In [240]:
# 구성한 Layer의 test
e = Post_processing(1024,to_embedding,Tout=tf.float64)(w)
for c in e:
    print(c)

tf.Tensor(
[ 0.20697494 -0.16078305  0.04480495 ... -0.18544388  0.67773646
  0.60726237], shape=(1024,), dtype=float64)
tf.Tensor(
[ 0.35158053 -0.53133172  0.05704678 ... -0.37279111  0.59129167
  0.58416045], shape=(1024,), dtype=float64)
tf.Tensor(
[ 0.19844982 -0.38411286  0.00347876 ... -0.39883673  0.88680857
  0.72472489], shape=(1024,), dtype=float64)


In [241]:
# 별로 중요하지는 않지만 Lambda layer를 활용하기 위한 assert 함수 구성
def assert_layer(input_data,out_dim=None):
    #tf.print(input_data)
    #print(input_data)
    assert input_data.shape[1] == out_dim
    return input_data

In [242]:
# 드디어 generator 구현
# 효과적으로 구성된 것인지는 모르겠음... 이것은 아직 많은 연구가 필요함.
# 또한 LSTM으로 바꾸어 길이의 한게를 극복해야 할 것...

def make_generator_model(org_words):
    input = Input(shape=(org_words,), dtype='float64') 
    x1 = Dense(org_words*2, use_bias=False)(input)
    x1 = LeakyReLU(0.2)(x1)
    x1 = Dense(org_words*4, use_bias=False)(x1)
    x1 = BatchNormalization()(x1)
 
    x1 = LeakyReLU(0.2)(x1)
    x1 = Dense(org_words*4*2, use_bias=False)(x1)
    x1 = BatchNormalization()(x1)
    x1 = Dense(org_words, use_bias=False,activation='tanh')(x1)
    x1 = Lambda(assert_layer,arguments={'out_dim':org_words})(x1)
    #x1 = Reshape((max_length, total_words))(x1)
    #x1 = BatchNormalization()(x1)
    #x1 = Softmax()(x1)        
    #x1 = MyCustomLayer(max_length*total_words)(x1)
    txt = Post_processing(0,to_text,Tout=tf.string)(x1)
    emb = Post_processing(1024,to_embedding,Tout=tf.float64)(x1)
    #cmr = Post_processing(0,to_compression_ratio,Tout=tf.float64)(x1)
    cod = Post_processing(128,to_morpcoding,Tout=tf.int32)(x1)
    
    #model = Model(input,[txt,emb,cmr,cod])
    model = Model(input,[txt,emb,cod])
    
    model.summary()
    return model

generator = make_generator_model(_NOISE_DIM)

Model: "functional_45"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_50 (InputLayer)           [(None, 301)]        0                                            
__________________________________________________________________________________________________
dense_75 (Dense)                (None, 602)          181202      input_50[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_47 (LeakyReLU)      (None, 602)          0           dense_75[0][0]                   
__________________________________________________________________________________________________
dense_76 (Dense)                (None, 1204)         724808      leaky_re_lu_47[0][0]             
______________________________________________________________________________________

In [243]:
# generator의 test
# Create a random noise and generate a sample
noise = tf.random.normal([3,_NOISE_DIM])
texts,embeddings,morpcodes = generator(noise, training=True)
print(texts.shape)
for i,txt in zip(range(len(texts)),texts):
    print(f" {i+1}> {txt.numpy().decode('utf-8')}" )
print(embeddings.shape)
print(compratios.shape)
print(morpcodes.shape)

(3,)
 1> 아기가 무럭무럭 소녀가 걱정되었어요.그래서 맞이했어요.새어머니는 데리고 고약한 그런데 집안일이 저애를 언니는 언니들을 무도회장으로 마법사 빙그레 너를 도마뱀을 구해 호박이 건드렸어요.그랬더니 구슬 바뀌웠어요.신데렐라 발을 구두를 되면 마부는 왕자님도 빼았겼어요.완자님은 추었어요.신데렐라는 놀랐어요. 신데렐라가 유리 가서 결혼하겠어요.그래서 구두를 유리 수 유리 꼭 왕궁으로 
 2> 여자 소녀의 어머니가 세상을 아버지는 두 못마땅했어요. 종일 도맡아 지칠때면난롯가에 앉아서 엄마 무도회가 고개를 내가 보내주마호박 한개와 두마리 건드리자 마차로 멋진 변했답니다. 예쁜 유리 밤 처음대로 바늗시 해. 아가씨들은 춤을 신데렐라는 신데렐라가 구두의 돌아다녔어요. 조용히 있나요?​신데렐라는 맞았어요. 왕궁으로 뒤 오래오래 
 3> 집에 자라서 되었어요.그러던 어느날 병이들어그만 말았어요. 나이가 새어머니와 예쁘고 이번에는 재투성이잖아요. 소녀를 놀려 떠났어요.신데렐라도 신데렐라는 시작했어요.신데렐라 할머니가 있었어요. 할머니가 화려한 황금 신데렐라의 빛나는 밤 열두시가 모든게 되기 알겠지? 왕자님과 줄도 벽시계가 왕궁을 구두를 층계에서 구두를 가서 유리 보았지만한눈에 신데렐라가 다가와 
(3, 1024)
(10,)
(3, 128)


# Discriminator 구현

먼저 요약을 구분하기 위한 discriminator_summ 구현

In [244]:
# 문장에 대한 embeddings를 이용하여 org_text_emb (org_text의 embedding)과의 유사도를 계산한다.
import scipy

@tf.custom_gradient
def to_similarity(w):

    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 10, 1e-3)  # gradient 증폭, 이래도 되는 건지....
        dy_arr = tf.reshape(dy,(dy.shape[0],1,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],1024),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],1024))
        return dy_arr_st

    similarities = []
    value = None
    try:
        for embedding in w:
            distances = scipy.spatial.distance.cdist([embedding], [org_text_emb], "cosine")[0]
            similarities.append(distances[0])
            
        value = tf.constant(similarities,dtype=tf.float64)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad


In [245]:
def make_discriminator_model():
    input_emb = Input(shape=(1024,), dtype='float64') 
    x1 = Post_processing(0,to_similarity,Tout=tf.float64)(input_emb)
    x1 = Reshape((1,))(x1)    
    #x1 = Dense(1024*2)(input_emb)
    #x1 = BatchNormalization()(x1)
    #x1 = LeakyReLU()(x1)

    
    #input_cmp = Input(shape=(), dtype='float64') 
    imput_mrp = Input(shape=(_MAX_MORP_LENGTH,), dtype='int32')

    #x2 = Reshape((1,))(input_cmp)
    x3 = Dense(256)(imput_mrp)
    #x3 = BatchNormalization()(x3)
    x3 = LeakyReLU()(x3)

    x3 = Dense(256*3)(x3)
    #x3 = BatchNormalization()(x3)
    x3 = LeakyReLU()(x3)

    #concatted = Concatenate(axis=1)([x1, x2, x3])
    concatted = Concatenate(axis=1)([x1, x3])
    x4 = Flatten()(concatted)
    #x4 = Dense(64, use_bias=False)(x4)
    #x4 = BatchNormalization()(x4)
    #x4 = LeakyReLU()(x4)
    x4 = Dense(1,activation='sigmoid')(x4)
    
    #model = Model([input_emb,input_cmp,imput_mrp],x4)
    model = Model([input_emb,imput_mrp],x4)
    model.summary()
    
    return model

discriminator = make_discriminator_model()

Model: "functional_47"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_52 (InputLayer)           [(None, 128)]        0                                            
__________________________________________________________________________________________________
dense_79 (Dense)                (None, 256)          33024       input_52[0][0]                   
__________________________________________________________________________________________________
input_51 (InputLayer)           [(None, 1024)]       0                                            
__________________________________________________________________________________________________
leaky_re_lu_49 (LeakyReLU)      (None, 256)          0           dense_79[0][0]                   
______________________________________________________________________________________

In [246]:
# discriminator test

predict = discriminator([embeddings,morpcodes])
print(predict)

tf.Tensor(
[[0.5961309 ]
 [0.78245354]
 [0.85123605]], shape=(3, 1), dtype=float32)


# GAN 을 이용한 loss 의 gradient 구현 --> 빡심

In [247]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [248]:
# 디짐
@tf.function
def train_step(real_embedding,real_morpcoding):
  
    # 1 - Create a random noise to feed it into the model
    # for the text generation
    noise = tf.random.normal([BATCH_SIZE, _NOISE_DIM])
    
    # 2 - Generate text and calculate loss values
    # GradientTape method records operations for automatic differentiation.
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        texts,embeddings,morpcodes = generator(noise, training=True)
        #embeddings = generator(noise, training=True)
        # real에 가까우려면 discriminator의 학습은 real_embedding 이 zero (0)에 가깝게 학습시켜야 함.
        # 하지만 압축율의 개념으로는 본래는 ones (1)가 맞음.
        real_output = discriminator([real_embedding,real_morpcoding], training=True)
        fake_output = discriminator([embeddings,morpcodes], training=True)

        #tf.print('train_step : embeddings.shape=',embeddings.shape)
        gen_loss = generator_loss(fake_output)
        #tf.print('train_step : gen_loss=',gen_loss)
        disc_loss = discriminator_loss(real_output, fake_output)
        #tf.print('train_step : disc_loss=',disc_loss)

    # 3 - Calculate gradients using loss values and model variables
    # "gradient" method computes the gradient using 
    # operations recorded in context of this tape (gen_tape and disc_tape).
    
    # It accepts a target (e.g., gen_loss) variable and 
    # a source variable (e.g.,generator.trainable_variables)
    # target --> a list or nested structure of Tensors or Variables to be differentiated.
    # source --> a list or nested structure of Tensors or Variables.
    # target will be differentiated against elements in sources.

    # "gradient" method returns a list or nested structure of Tensors  
    # (or IndexedSlices, or None), one for each element in sources. 
    # Returned structure is the same as the structure of sources.
    
    gradients_of_discriminator = disc_tape.gradient(disc_loss, 
                                                discriminator.trainable_variables)
    #tf.print('train_step : gradients_of_discriminator=',gradients_of_discriminator)   
    noise = tf.random.normal([BATCH_SIZE, _NOISE_DIM])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        texts,embeddings,morpcodes = generator(noise, training=True)
        #embeddings = generator(noise, training=True)
        #real_output = discriminator(real_embedding, training=True)
        fake_output = discriminator([embeddings,morpcodes], training=True)

        #tf.print('train_step : embeddings.shape=',embeddings.shape)
        gen_loss = generator_loss(fake_output)
        #tf.print('train_step : gen_loss=',gen_loss)
        #disc_loss = discriminator_loss(real_output, fake_output)
        #tf.print('train_step : disc_loss=',disc_loss)

    gradients_of_generator = gen_tape.gradient(gen_loss, 
                                               generator.trainable_variables)
    #tf.print('train_step : gradients_of_generator=',gradients_of_generator)
 

    # 4 - Process  Gradients and Run the Optimizer
    # "apply_gradients" method processes aggregated gradients. 
    # ex: optimizer.apply_gradients(zip(grads, vars))
    """
    Example use of apply_gradients:
    grads = tape.gradient(loss, vars)
    grads = tf.distribute.get_replica_context().all_reduce('sum', grads)
    # Processing aggregated gradients.
    optimizer.apply_gradients(zip(grads, vars), experimental_aggregate_gradients=False)
    """
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    #tf.print('train_step : after discriminator_optimizer')    

In [249]:
EPOCHS = 5
# 요약문 생성의 확인을 위해 10개의 문장을 생성하고 train과정에서 각 epoch마다 변화를 확인한다.
seed = tf.random.normal([10, _NOISE_DIM])

In [250]:
# 생성된 문장의 원문 유사도를 측정하기 위한 함수

import scipy
#print(doc_emb)
def similarity_score(queries,org_embedding):

    total_score = 0
    query_embeddings = embedder.encode(queries,show_progress_bar=False)
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], [org_embedding], "cosine")[0]
        results = zip(range(len(distances)), distances)
        for idx, distance in results:
            total_score += 1-distance
    return total_score

queries = []

texts,embeddings,morpcodes = generator(seed,training=False)

#count = 0
for t in texts:
    summary_text = t.numpy().decode('utf-8')
    queries.append(summary_text)
print('Similarity score:',str(similarity_score(queries,org_text_emb)))


Similarity score: 5.708030546274871


In [251]:
# Print iterations progress
class ProgressBar:
    # pb = ProgressBar(total=20, prefix = 'Epoch 1')
    def __init__(self,total=20, prefix = '', suffix = '', decimals = 1, length = 20, fill = '█', printEnd = "\r"):
        self.total = total
        self.prefix = prefix
        self.suffix = suffix
        self.decimals = decimals
        self.length = length
        self.fill = fill
        self.printEnd = printEnd
        self.ite = 0
    # pb.printProgress(1,'~~~~')
    def printProgress(self,iteration, text):
        self.ite += iteration
        percent = ("{0:." + str(self.decimals) + "f}").format(100 * (self.ite / float(self.total)))

        filledLength = int(self.length * self.ite // self.total)
        bar = self.fill * filledLength + '-' * (self.length - filledLength)
        print(f'\r{self.prefix} |{bar}| {percent}% {self.suffix}  {text}', end="", flush=True)
        # Print New Line on Complete
        if self.ite == self.total: 
            print()

In [252]:
import time
from IPython import display # A command shell for interactive computing in Python.
import re

def train(dataset, epochs):
    print('Start with seed text.')
    seed = tf.random.normal([10, _NOISE_DIM])
    print('-------------------------------------------------------')
    texts,embeddings,morpcodes = generator(seed,training=False)
    for i,t in  zip(range(len(texts)),texts):
        summary_text = t.numpy().decode('utf-8')
        print(f'{i+1} > {summary_text}')        
    print('-------------------------------------------------------')
    
    predict = discriminator([embeddings,morpcodes])
    socre = 0
    for s in predict:
        socre += s.numpy()
    print(f'Initial score:{socre}')
    print(' ')

    # A. For each epoch, do the following:
    for epoch in range(epochs):
        start = time.time()
        pb = ProgressBar(total=BATCH_COUNT, prefix = f'Epoch:{str(epoch+1)}/{epochs}')
        pb.printProgress(0,'Start batch.')
        # 1 - For each batch of the epoch, 
        for batch_num,(emb_batch_set,cod_batch_set) in zip(range(len(dataset)),dataset):
            # 1.a - run the custom "train_step" function
            # we just declared above
            #print(image_batch.shape)
            train_step(emb_batch_set,cod_batch_set)
            texts,embeddings,morpcodes = generator(seed,training=False)
            predict = discriminator([embeddings,morpcodes])
            socre = 0
            for s in predict:
                socre += s.numpy()
            pb.printProgress(+1,f"Time for batch {batch_num + 1}/{BATCH_COUNT} is {int(time.time()-start)} sec, score:{socre}, text:{texts[0].numpy().decode('utf-8')}")
        # 4 - Print out the completed epoch no. and the time spent
        #print (f'Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
        #texts,embeddings,compratios,morpcodes = generator(seed,training=False)
        count = 0
        queries = []
        texts,embeddings,morpcodes = generator(seed,training=False)
        for i,t in  zip(range(len(texts)),texts):
            summary_text = t.numpy().decode('utf-8')
            print(f'{i+1} > {summary_text}')
            queries.append(summary_text)
            c = [m.start() for m in re.finditer(_MISMATCH_WORD, summary_text)]
            count += len(c)
        predict = discriminator([embeddings,morpcodes])
        socre = 0
        for s in predict:
            socre += s.numpy()            
        print(f'Discriminator score:{socre} Similarity score:{str(similarity_score(queries,org_text_emb))}')
        print('')

In [253]:
if DO_ALL==False:
    EPOCHS = 30
    train(dataset, EPOCHS)

Start with seed text.
-------------------------------------------------------
1 > 마음씨 소녀가 되었어요.그러던 말았어요. 소녀의 소녀가 걱정되었어요.그래서 데리고 소녀가 못마땅했어요. 쉬곤 했지요. 집에도 초대장이 왔어요. 데리고 무도회에 싶니?신데렐라가 빙그레 지팡이로 생쥐와 반짝반짝 열두시가 돼.그러니까 아름다운 모인 다른 추었어요.신데렐라는 추느라 땡...... 놀랐어요. 한 벗겨졌어요.하지만 유리 온 유리 그때 볼 수 있나요?​신데렐라는 
2 > 집에 태어났어요.아기는 예쁘고 아버지는 나이가 심술쟁이들이었어요.새어머니는 게 엄마 열렸어요.신데렐라의 집에도 왔어요. 남은 싶니?신데렐라가 할머니가 무도회에 한개와 생쥐 도마뱀을 구해 건드리자 호박이 화려한 생쥐와 생쥐는 옷도 내밀어 열두시가 되면 밤 해. 아름다운 추느라 알리는 구두 결혼하겠어요.그래서 유리 보았지만한눈에 구두는 그때 한번 
3 > 태어났어요.아기는 무럭무럭 어느날 말았어요. 아버지는 나이가 못마땅했어요. 이번에는 날 집에도 초대장이 싶니?신데렐라가 마법사 빙그레 너를 무도회에 외웠어요.그리고 호박을 화려한 변했어요.이번에는 신데렐라에게 호박으로 도마뱀으로 아름다운 다른 가는 소리에 화들짝 구두 유리 돌아다녔어요. 구두는 너무 그때 말했어요.저도 수 있나요?​신데렐라는 신하게 건넨 구두는 
4 > 집에 예쁘고 아버지는 나이가 데리고 왔어요.그러나 게 했지요. 집에도 왔어요. 남은 울기 시작했어요.신데렐라 빙그레 보내주마호박 두마리 주문을 호박이 생쥐는 마부로 옷도 신데렐라에게 빛나는 신겨 모든게 흰말은 아름다운 마음을 아가씨들은 줄도 시를 구두 결혼하겠어요.그래서 유리 구두는 그때 수 구두는 신데렐라의 그 
5 > 예쁘고 어느날 말았어요. 소녀의 아버지는 두 왔어요.그러나 게 저애를 재투성이잖아요. 무도회가 초대장이 왔어요. 남은 신데렐라는 마법사 할머니가 빙그레 너를 외웠어요.그리고 생쥐는 신데렐라에게 처음대로 아름다운 신데

KeyboardInterrupt: 

In [ ]:
org_term_set = org_text.split(' ')

In [ ]:
len(org_term_set)

In [ ]:
## 그냥 테스트....

def text_gen(w):
    s_w = tf.sort([w],axis=-1,direction='DESCENDING')
    #print(s_w)
    #for w_0r,s_w_0r in zip(w,s_w):
    w_0r = w
    s_w_0r = s_w[0]
    text = ""
    wh = tf.where(tf.math.greater_equal(w_0r,s_w_0r[_MAX_GEN_TOKEN]))[:_MAX_GEN_TOKEN]
    order = tf.reshape(wh,(_MAX_GEN_TOKEN,)).numpy()
    for k in order:
        text += word_table[k]+' '    
    return text,order 
                

noise = tf.random.normal([_NOISE_DIM])    
txt, ord = text_gen(noise)
print(txt)
print(ord)
'''
for i in range(10):
    noise = tf.random.normal([1,_NOISE_DIM])
    texts,embeddings,compratios,morpcodes = generator(noise, training=True)
    p_score = 0
    for txt in texts:
        embedding = embedder.encode([txt.numpy().decode('utf-8')],show_progress_bar=False)
        distances = scipy.spatial.distance.cdist(embedding, [org_text_emb], "cosine")[0]
        score = 1-distances[0]
        reward = p_score - score
        if p_score == 0:
            reward = -reward
        print(f'score:{score} reward:{reward}') # 1-distance
        p_score = score
'''
pass